In [1]:
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

In [2]:
import sys
sys.path.append('../../code')
import visualize
from crispr import CasTemplate
from pipeline import NDNFPipeline

In [3]:

cas_templates = [CasTemplate(pam_motif = 'NNGRRT', typ='II', 
                             protospacer_length=20, name='SaCas9-WT'),
                 CasTemplate(pam_motif = 'NNNRRT', typ='II', 
                             protospacer_length=20, name='SaCas9-KKH-N'),
                 CasTemplate(pam_motif = 'NNCRRT', typ='II', 
                             protospacer_length=20, name='SaCas9-KKH-C'),
                 CasTemplate(pam_motif = 'NNTRRT', typ='II', 
                             protospacer_length=20, name='SaCas9-KKH-T'),
                 CasTemplate(pam_motif = 'NNARRT', typ='II', 
                             protospacer_length=20, name='SaCas9-KKH-A'),
                 CasTemplate(pam_motif = 'NGG', typ='II', 
                             protospacer_length=20, name='SpCas9-WT'),
                 CasTemplate(pam_motif = 'NGAG', typ='II', 
                             protospacer_length=20, name='SpCas9-EQR'),
                 CasTemplate(pam_motif = 'NGA', typ='II', 
                             protospacer_length=20, name='SpCas9-VQR'),
                 CasTemplate(pam_motif = 'NGCG', typ='II', 
                             protospacer_length=20, name='SpCas9-VRER'),
                 CasTemplate(pam_motif = 'NG', typ='II', 
                             protospacer_length=20, name='SpCas9-NG'),
                 CasTemplate(pam_motif = 'NRN', typ='II', 
                             protospacer_length=20, name='SpRY'),
                 CasTemplate(pam_motif = 'TTTV', typ='V', 
                             protospacer_length=20, name='Cpf1-TTTV'),
                 CasTemplate(pam_motif = 'TTTN', typ='V', 
                             protospacer_length=20, name='Cpf1-TTTN'),
                 CasTemplate(pam_motif = 'TYCV', typ='V', 
                             protospacer_length=20, name='Cpf1-TYCV'),
                 CasTemplate(pam_motif = 'TATV', typ='V', 
                             protospacer_length=20, name='Cpf1-TATV'),
                 CasTemplate(pam_motif = 'TTTR', typ='V', 
                             protospacer_length=20, name='Cpf1-TTTR'),
                ]
                 

In [4]:
# Path to a collection of HIV sequences to use as the nomination set.
NOMINATE_FASTA = '../../data/nominate.fasta'

# Path to a collection of HIV sequences to use as the validation set.
VALIDATE_FASTA = '../../data/test.fasta'

# Path to a single HIV sequence to use as the common reference position set.
REFERENCE_FASTA = '../../data/hxb2.fasta'

# Path to an off-target reference genome.
OFFTARGET_REF = '../../data/GRCh38.fna'

# The number of protospacers to extract from the nomination set.
TOP_N_KMERS = 20_000

# The target number of protospacers after diversification.
DIVERSITY_TARGET = TOP_N_KMERS*2

MISMATCH = 2
BROAD_SPECTRUM = 0.9
MAX_OFF = 0

In [5]:
with open(NOMINATE_FASTA) as handle:
    nomination_seqs = list(SeqIO.parse(handle, 'fasta'))
num_nom = len(nomination_seqs)
len_nom = sum(len(seqR.seq) for seqR in nomination_seqs)/1_000_000
print(f'Loading {num_nom} sequences with a total of {len_nom:0.2f} Mbp for nomination.')

Loading 3228 sequences with a total of 28.73 Mbp for nomination.


In [6]:
MUT_DATA = pd.read_csv('../../data/mutation_scores.csv')

def rc_scores(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    mean_rc = MUT_DATA.loc[mask, 'RC Index'].mean()
    
    return mean_rc

def lethal_count(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    num_lethal = (MUT_DATA.loc[mask, 'Effect'] == "lethal").sum()
    
    return num_lethal

def process_cas(cas):
    
    nominated, protospacer_counts = NDNFPipeline.nominate(cas, nomination_seqs, 
                                                          top_n=TOP_N_KMERS, 
                                                          return_counts=True)
    diversed = nominated.diversify(DIVERSITY_TARGET)
    narrowed, protospacer_hist_freq = diversed.narrow(VALIDATE_FASTA, min_rate=0.5, return_freqs=True,
                                                      mismatch=MISMATCH)
    good_hits, off_target_counts = narrowed.filter(OFFTARGET_REF, 
                                               mismatch = MISMATCH, 
                                               return_counts=True)
    ref_diversed = diversed.reference_map(REFERENCE_FASTA, mismatch=3, enforce_pam=False)
    ref_diversed['on_target_freq'] = ref_diversed['protospacer'].map(protospacer_hist_freq.get)
    ref_diversed['off_target_hits'] = ref_diversed['protospacer'].map(off_target_counts.get)
    ref_diversed['num_lethal'] = ref_diversed['start'].map(lethal_count)
    ref_diversed['mean_rc'] = ref_diversed['start'].map(rc_scores)
    ref_diversed['cas'] = cas.name
    ref_diversed['pam_motif'] = cas.pam_motif
    return ref_diversed
    

In [7]:
for cas in cas_templates:
    print('Processing', cas.name)
    fname = f'{cas.name}.info.csv'
    if not os.path.exists(fname):
        process_cas(cas).to_csv(fname, index=False)


Processing SaCas9-WT
Processing SaCas9-KKH-N
Processing SaCas9-KKH-C
Processing SaCas9-KKH-T
Processing SaCas9-KKH-A
Processing SpCas9-WT
Processing SpCas9-EQR
Processing SpCas9-VQR
Processing SpCas9-VRER
Processing SpCas9-NG
Processing SpRY
Processing Cpf1-TTTV


100%|██████████| 40000/40000 [07:08<00:00, 93.41it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
59.678 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
62.7988 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
30.8564 seconds elapsed.


Processing Cpf1-TTTN


100%|██████████| 40000/40000 [07:30<00:00, 88.80it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
72.6244 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
153.579 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
35.5842 seconds elapsed.


Processing Cpf1-TYCV


100%|██████████| 40000/40000 [06:55<00:00, 96.16it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
58.7468 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
92.6713 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
35.9822 seconds elapsed.


Processing Cpf1-TATV


100%|██████████| 40000/40000 [06:48<00:00, 97.82it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
53.7396 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
45.3329 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
34.6202 seconds elapsed.


Processing Cpf1-TTTR


100%|██████████| 40000/40000 [06:48<00:00, 98.03it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
39.4866 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
45.798 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
36.1562 seconds elapsed.
